Sentiment Analysis of Employee Text Messages

In [1]:
import pandas as pd

#importing the provided dataset from downloads
df = pd.read_csv(r"C:\Users\USER\Downloads\test(in).csv",header=0)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\

AttributeError: _ARRAY_API not found

I am doing some preliminary data cleaning before proceeding with the data. The dataset contains no missing values or duplicate values. All the data in repective header column is object type. The date column type was changed to datetime64 format. Since there were no missing values and no error is raised, all date strings are valid and convertable.

In [2]:
missing_values = df.isnull().sum()
print(missing_values)
print()

duplicates = df.duplicated().sum()
print("No. of duplicate data =", duplicates)
print()

# to check data type of each coloumn
#print(df.dtypes)           
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dtype)

Subject    0
body       0
date       0
from       0
dtype: int64

No. of duplicate data = 0

datetime64[ns]


Sentiment Labelling

In this case there is no need for data processing. We will be using Textblob which is an easy tool for sentiment analysis. There is no need to tokenize the data. Textblob automatically handles that. There is also need to remove stop words because textblob works best on full sentences. Removing them can hurt accuracy of model because we might remove context. Sentiment analysis can also be done using other models like BERT. Textblob is used because of ease of use and low computational demand. If you want more accurate results BERT must be used. It outperforms textblob textblob due to its better ability to learn complex patterns form large amounts of data.

In [3]:
from textblob import TextBlob

def sentiment_analysis(text):
    analysis = TextBlob(str(text))
    score = analysis.sentiment.polarity
    if score>0:
        return "Positive"
    elif score<0:
        return "Negative"
    else:
        return "Neutral"

df['sentiment labelling'] = df['body'].apply(sentiment_analysis)
print(df['sentiment labelling'])

0       Positive
1       Negative
2        Neutral
3       Negative
4       Negative
          ...   
2186    Positive
2187    Positive
2188    Positive
2189    Positive
2190     Neutral
Name: sentiment labelling, Length: 2191, dtype: object


Exploratory Data Analysis (EDA) -

In [4]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(df['sentiment labelling'])
plt.ylabel('Distribution of Sentiment')
plt.show()

# let's check the volume of messages received or send in year 2010 and 2011
year_2010 = df[df['date'].dt.year == 2010]
year_2011 = df[df['date'].dt.year == 2011]
print("No. of texts send in 2010:",len(year_2010))
print("No. of texts send in 2011:",len(year_2011))

#We also want to check who made the most signicant and least contribution
contribution = df['from'].value_counts()
print(f'Most Contributed by {contribution.keys()[0]}\nLeast contributed by {contribution.keys()[-1]}')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda3\Lib\site-packages\

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

As seen from the graphs, the no. of positive sentiments seen across messages is way higher than neutral or negative, with fewer negative than neutral ones. The volume of text messages sent each year is almost the same. The overall sentiment distribution doesn't change over the year.

Employee Score Calculation

In [5]:
#Assigning a score to each message. +1 if sentiment is positive, -1 for negative, or else 0 for neutral.
def score(sentiment):
    if sentiment == "Positive":
        return int(1)
    elif sentiment == "Negative":
        return int(-1)
    else:
        return int(0)

df['score'] = df['sentiment labelling'].apply(score)

#before we can compute the monthly sentiment score of each employee, we need to separate data by month(and year) or label them.

df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

#for each employee in a year per month we sum the monthly scores.
#we don't need to reset counter. There is a different counter for each month.
monthly_score = df.groupby(['from', 'year', 'month'])['score'].sum().reset_index()
print(monthly_score)

                            from  year  month  score
0    bobette.riner@ipgdirect.com  2010      1      2
1    bobette.riner@ipgdirect.com  2010      2      8
2    bobette.riner@ipgdirect.com  2010      3      4
3    bobette.riner@ipgdirect.com  2010      4      4
4    bobette.riner@ipgdirect.com  2010      5      2
..                           ...   ...    ...    ...
235         sally.beck@enron.com  2011      8      7
236         sally.beck@enron.com  2011      9      1
237         sally.beck@enron.com  2011     10      4
238         sally.beck@enron.com  2011     11      5
239         sally.beck@enron.com  2011     12      2

[240 rows x 4 columns]


Employee Ranking

In [6]:
#Now to get the 3 employees withe best and worst scores

#sorting year and month in chronological order. Then score from highest to lowest. It is already sorted in alphabetically
monthly_sorted = monthly_score.sort_values(['year','month', 'score'], ascending=[True, True, False])

#for a given year the function outputs the top three employees each month
def top_rank(year):
    rank = []
    for i in range(1,13):
        monthly_rank = monthly_sorted[(monthly_sorted['year']==year) & (monthly_sorted['month']==i)].iloc[0:3]
        rank.append(monthly_rank)

    return pd.concat(rank)
    
#outputs three employees with the lowest score each month in a given year.       
def bottom_rank(year):
    rank = []
    for i in range(1,13):
        monthly_rank = monthly_sorted[(monthly_sorted['year']==year) & (monthly_sorted['month']==i)].iloc[-3:]
        rank.append(monthly_rank)

    return pd.concat(rank)

display(bottom_rank(2010))

,from,year,month,score
96,johnny.palmer@enron.com,2010,1,2
216,sally.beck@enron.com,2010,1,2
192,rhonda.denton@enron.com,2010,1,0
145,lydia.delgado@enron.com,2010,2,1
169,patti.thompson@enron.com,2010,2,1
217,sally.beck@enron.com,2010,2,0
170,patti.thompson@enron.com,2010,3,1
50,eric.bass@enron.com,2010,3,0
194,rhonda.denton@enron.com,2010,3,0
219,sally.beck@enron.com,2010,4,3


Flight Risk Identification

In [7]:

#Using a sub dataset. taking dataset where the sentiment is only negative

df1 = df[df['sentiment labelling'] == 'Negative']
employees = df1['from'].unique()
def risk(year):
    at_risk = []
    df2 = df1[df1['year'] == year]
    for i in range(1,13):
        for employee in employees:
            stats = df2[(df2['from'] == employee) & (df2['month'] == i)]
            if len(stats['score']) >= 4:
                at_risk.append(employee)
    return at_risk

print("At risk employees include :\n", risk(2010))

At risk employees include :
 ['johnny.palmer@enron.com', 'don.baughman@enron.com', 'john.arnold@enron.com', 'sally.beck@enron.com', 'rhonda.denton@enron.com', 'eric.bass@enron.com', 'bobette.riner@ipgdirect.com', 'rhonda.denton@enron.com']


Predictive Modeling

A simple linear regression model is used to analyze sentiment trends and predict future sentiment scores.

In [8]:
!pip install --upgrade numpy scikit-learn

In [11]:
#important features include frequency of text messages each month, total sentiment score each month.

features = df[['body', 'month', 'year', 'score']]
freq = []
tot_score = []
months = []
for year in [2010,2011]:
    for month in range(1,13):
        feature_df = features[(features['year'] == year) & (features['month'] == month)]
        freq.append(len(feature_df['body']))
        tot_score.append(feature_df['score'].sum())
        months.append(month)

#prepering data to be used in model
model_df = pd.DataFrame({
    'month': months,
    'message_freq': freq,
    'total_sentiment': tot_score
})

x = model_df[['month','message_freq']]
y = model_df['total_sentiment']

[np.int64(37), np.int64(45), np.int64(33), np.int64(37), np.int64(39), np.int64(40), np.int64(33), np.int64(34), np.int64(44), np.int64(36), np.int64(29), np.int64(28), np.int64(40), np.int64(45), np.int64(37), np.int64(40), np.int64(41), np.int64(46), np.int64(49), np.int64(44), np.int64(47), np.int64(43), np.int64(44), np.int64(37)]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

#splitting data into testing and training data set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state = 42)

#initializing and fitting the model
model = LinearRegression()
model.fit(x_train,y_train)

#predict and evaluate
y_pred = model.predict(x_test)

#to test the performance of our model 
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error=",rmse)

r2 = r2_score(y_test, y_pred)
print(r2)

Root Mean Squared Error= 6.931103073725088
0.042260968528681


The r2 score, meaning the goodness of fit, comes out to be 0.042 which is as good which means the regresssion model poor explains the variability in the dependant variable. It shows almost no correlation. To improve the performance, instead of linear regression model, we can use a polynomial regression model to find more complicated relation in the data. The problem may also have risen from not identifying important features in the data.

The root mean square error tells you the average magnitude of the differences between predicted values and actual values in a dataset. Since the total sentiment score per month averages around 45, a rmse of 6.9 is not that bad.